In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder \
        .appName('mllib') \
        .getOrCreate()

In [0]:
storage_account_name = "chengyangbde"
storage_account_access_key = "89Y9ZLqYL8RN3MNqcTnvMGNnPOuArORjnNDbWLhkXzkRTYaD/GfvSmaY8sdFz+wonSC4llRg5osD+AStJmIHug=="
blob_container_name = "bde-2"

In [0]:
mount_points = [mount.mountPoint for mount in dbutils.fs.mounts()]

if f"/mnt/{blob_container_name}/" not in mount_points:
    dbutils.fs.mount(
      source = f'wasbs://{blob_container_name}@{storage_account_name}.blob.core.windows.net',
      mount_point = f'/mnt/{blob_container_name}/',
      extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net': storage_account_access_key}
    )
else:
    print(f"/mnt/{blob_container_name}/ is already mounted. Skipping.")

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, lit, month, year

# Initialize Spark session
spark = SparkSession.builder.appName("Taxi Fare Prediction").getOrCreate()

# Read the dataset from the Parquet file
parquet_path = "/mnt/bde-2/combined_data_cleaned3.parquet"
df = spark.read.parquet(parquet_path)

# Compute trip duration in seconds
df = df.withColumn("trip_duration", (col("tpep_dropoff_datetime").cast("long") - col("tpep_pickup_datetime").cast("long")))

# Fill null values with defaults
default_values = {
    "trip_distance": 0.0,
    "passenger_count": 1,
    "RatecodeID": 1,
    "trip_duration": 0
}
df = df.fillna(default_values)

# Separate data for test set before sampling
test_df = df.filter((year("tpep_pickup_datetime") == 2022) & (month("tpep_pickup_datetime") >= 10))

# Sample data for training and validation
df_sampled = df.sample(False, 0.6, seed=42)

# Split the sampled data into training and validation
train_df, validation_df = df_sampled.randomSplit([0.8, 0.2], seed=42)

# Load the Q3C answer from the CSV file
q3c_df = spark.read.csv("/mnt/bde-2/Q3Canswer.csv", header=True, inferSchema=True)
average_fare_q3c = q3c_df.collect()[0][0]

# Vectorize the features without the location IDs
features = ["trip_distance", "passenger_count", "RatecodeID", "trip_duration"]
vector_assembler = VectorAssembler(inputCols=features, outputCol="features", handleInvalid="keep")

# Decision Tree Regression Model
dt = DecisionTreeRegressor(featuresCol="features", labelCol="total_amount")
pipeline_dt = Pipeline(stages=[vector_assembler, dt])
dt_model = pipeline_dt.fit(train_df)
dt_predictions = dt_model.transform(validation_df)
dt_rmse = evaluator.evaluate(dt_predictions)
print(f"Decision Tree Regression Model RMSE on Validation Data: {dt_rmse:.2f}")

dt_predictions_test = dt_model.transform(test_df)
dt_rmse_test = evaluator.evaluate(dt_predictions_test)
print(f"Decision Tree Regression Model RMSE on Test Data: {dt_rmse_test:.2f}")

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2749248832212531>, line 49
     47 dt_model = pipeline_dt.fit(train_df)
     48 dt_predictions = dt_model.transform(validation_df)
---> 49 dt_rmse = evaluator.evaluate(dt_predictions)
     50 print(f"Decision Tree Regression Model RMSE on Validation Data: {dt_rmse:.2f}")
     52 dt_predictions_test = dt_model.transform(test_df)

NameError: name 'evaluator' is not defined

In [0]:
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction", metricName="rmse")
dt_rmse = evaluator.evaluate(dt_predictions)
print(f"Decision Tree Regression Model RMSE on Validation Data: {dt_rmse:.2f}")

dt_predictions_test = dt_model.transform(test_df)
dt_rmse_test = evaluator.evaluate(dt_predictions_test)
print(f"Decision Tree Regression Model RMSE on Test Data: {dt_rmse_test:.2f}")

Decision Tree Regression Model RMSE on Validation Data: 127.22
Decision Tree Regression Model RMSE on Test Data: 7.22
